In [1]:
#import matplotlib
%matplotlib qt4
#%matplotlib nbagg


In [2]:
import hyperspy.api as hs
import matplotlib.pyplot as plt

In [3]:
import numpy as np

In [325]:
def gauss(sigma, eRange):
    dE = eRange[1]-eRange[0]
    gx = np.arange(-3*sigma,3*sigma, dE)
    gaussian = np.exp(-0.5*(gx/sigma)**2)
    gaussian = gaussian/np.sum(gaussian)
    
    gauss =np.zeros((len(gaussian),1,1,1))
    gauss[:,0,0,0] = gaussian
    return gauss

In [327]:
def smooth(hist, eRange, sigma):
    gaussian = gauss(sigma, eRange)
    
    crop_front = len(gaussian)//2
    if len(gaussian)%2 == 1:
        crop_end = crop_front
    else:
        crop_end = crop_front-1
        
    return signal.convolve(hist, gaussian)[crop_front:-crop_end]

In [4]:
# Plot the DF, draw a ROI. Maneuver/resize it and close the window.
def set_ROI(s, shape="circle"):
    """ Selects an interactive region of interst to the signal
    :param s: t
    """
    import hyperspy.api as hs
 
    if s.axes_manager.navigation_dimension < 2:
        axes = "sig"
        x_axis = s.axes_manager[s.axes_manager.signal_indices_in_array[1]]
        y_axis = s.axes_manager[s.axes_manager.signal_indices_in_array[0]]
    else:
        axes = "nav"
        x_axis = s.axes_manager[s.axes_manager.navigation_indices_in_array[1]]
        y_axis = s.axes_manager[s.axes_manager.navigation_indices_in_array[0]]


    if shape == "circle":
        x = x_axis.axis[round(x_axis.size/2)]
        y = y_axis.axis[round(y_axis.size/2)]

        r_outer = x_axis.axis[round(3*x_axis.size/4)]
    
        sroi = hs.roi.CircleROI(x, y, r=r_outer)
        s.plot()
        sroi= sroi.interactive(s) 
        ss = hs.interactive(f=sroi.sum, event=sroi.events.data_changed)
        
    elif shape == "ring":
        x = x_axis.axis[round(x_axis.size/2)]
        y = y_axis.axis[round(y_axis.size/2)]

        r_outer = x_axis.axis[round(4*x_axis.size/5)]
        r_inner = x_axis.axis[round(3*x_axis.size/4)]
    
        sroi = hs.roi.CircleROI(x, y, r=r_outer, r_inner=r_inner)
        s.plot()
        sroi= sroi.interactive(s) 
        ss = hs.interactive(f=sroi.sum, event=sroi.events.data_changed)
    else:
        if not shape == "rectangle":
            print("Did not recognize shape, using rectangle")
        x1 = x_axis.axis[1]
        x2 = x_axis.axis[round(x_axis.size/10)]
        y1 = y_axis.axis[1]
        y2 = y_axis.axis[round(y_axis.size/10)]

        sroi = hs.roi.RectangularROI(x1, y1, x2, y2)
        s.plot()
        roi_signal = sroi.interactive(s)
        ss = hs.interactive(f=roi_signal.sum, event=roi_signal.events.data_changed) 
        
        
    return sroi, ss

In [277]:
def signal_by_q(s, q_inner, q_outer):
    """Centers a hollow solid sphere in diffraction space and add signal from whidthin the solid
    :param s: signal1D with 3D diffraction space
    :param q_inner: inner radius of hollow solid sphere (defines the hollow sphere within)
    :param q_outer: outer radius of hollow solid sphere (defines the solid range outside inner sphere)
    :returns: the added signal from the hollow solid sphere
    """
    # Center coordinate
    x, y, = 0, 0
    
    # Get axes
    z_axis = s.axes_manager[s.axes_manager.navigation_indices_in_array[2]]
    x_axis = s.axes_manager[s.axes_manager.navigation_indices_in_array[1]]
    y_axis = s.axes_manager[s.axes_manager.navigation_indices_in_array[0]]
    E_axis = s.axes_manager.signal_axes[0]
    
    # Get shape of EEL spectra
    E_shape = s.axes_manager.signal_shape[0]

    # Crate output signal
    sroi_signal = s.inav[0,0,0]
    sroi_signal.data = np.zeros(E_shape)
    
    # Loop through one axe, select the corresponding 2D region of interst in the other two axes
    for h in range(s.axes_manager.navigation_axes[-1].size):
        r_inner = None
        r_outer = None
        if abs(z_axis.axis[h]) <= abs(q_outer):
            r_outer = np.sqrt(q_outer**2-z_axis.axis[h]**2)
        if abs(z_axis.axis[h]) <= abs(q_inner):
            r_inner = np.sqrt(q_inner**2-z_axis.axis[h]**2)
        
        if r_outer:
            s_temp = s.inav[:,:,h]
            if r_inner:
                sroi = hs.roi.CircleROI(x, y, r=r_outer, r_inner=r_inner)
            else:
                sroi = hs.roi.CircleROI(x, y, r=r_outer)
            s_temp_signal = sroi(s_temp)
            sroi_signal += s_temp_signal.sum()
    return sroi_signal
    

In [328]:
p = hs.load("../Examples/direct.hspy")
p = p.transpose((2,1,0))
p = p.transpose()
p

<Signal1D, title: , dimensions: (51, 51, 51|80)>

In [329]:
p.axes_manager

<Axes manager, axes: (51, 51, 51|80)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
             q_a |     51 |      0 |    -0.5 |    0.02 |     a* 
             q_b |     51 |      0 |    -0.5 |    0.02 |     b* 
             q_c |     51 |      0 |    -0.5 |    0.02 |     c* 
---------------- | ------ | ------ | ------- | ------- | ------ 
     Energy Loss |     80 |        |       3 |   0.051 |     eV

In [330]:
pf = hs.load("../Examples/indirect_x.hspy")
pf = pf.transpose((2,1,0))
pf = pf.transpose()
pf

<Signal1D, title: , dimensions: (51, 51, 51|80)>

In [332]:
p_roi, p_roi_s = set_ROI(p)
pf_roi, pf_roi_s = set_ROI(pf)

In [303]:
p2d = p.as_signal2D((0,1))
p2d.plot()

In [338]:
hs.plot.plot_spectra([pf_roi_s, p_roi_s])